In [ ]:
!pip install transformers accelerate json faiss-cpu

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [18]:
import faiss
import json

index = faiss.read_index("vector_index.faiss")

with open("chunks_metadata.json", "r") as f:
    chunks = json.load(f)

print(f"Index contains {index.ntotal} vectors.")

Index contains 378 vectors.


In [19]:
from sentence_transformers import SentenceTransformer

model_embed = SentenceTransformer("all-MiniLM-L6-v2")

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [20]:
def retrieve_top_k_chunks(question, k=5):
    question_embedding = model_embed.encode([question], convert_to_numpy=True)
    distances, indices = index.search(question_embedding, k)
    top_chunks = [chunks[i] for i in indices[0]]
    return top_chunks

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [22]:
from transformers import pipeline

qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [27]:
question = "Does Medicare cover hearing aids?"

# Retrieve top chunks
top_chunks = retrieve_top_k_chunks(question, k=3)

# Build context from chunks
context = "\n\n".join(
    [f"(Page {chunk['page_num']}): {chunk['chunk']}" for chunk in top_chunks]
)

# Strong prompt instruction
prompt = f"""You are a medical document assistant.

Use the context below to answer the user's question. You must:
- Answer ONLY using the context.
- Clearly mention the relevant page numbers (e.g., "As stated on Page 7...").
- Do NOT make up or guess anything.
- Add a medical disclaimer: "This is not medical advice."

Context:
{context}

Question: {question}

Answer:"""

# Run the model
result = qa_pipeline(prompt, max_new_tokens=150)[0]['generated_text']

# Add fallback page numbers
page_nums = set(chunk['page_num'] for chunk in top_chunks)
page_refs = ", ".join(f"Page {p}" for p in sorted(page_nums))

final_answer = f"{result.strip()}\n Source(s): {page_refs}"
print(final_answer)

Medicare doesn’t cover hearing aids or exams for fitting them.
 Source(s): Page 42, Page 51, Page 56
